In [1]:
from pathlib import Path
import pandas as pd

In [2]:
folder_path = Path('data')
zip_files = list(folder_path.glob('*.zip'))
print(zip_files)
zip_file = zip_files[1].absolute()
print(zip_file)

[WindowsPath('data/2023-04-26_00-54-lenta.zip'), WindowsPath('data/2023-04-27_14-27-lenta.zip')]
c:\works\work_002\data\2023-04-27_14-27-lenta.zip


In [3]:
df = pd.read_csv(zip_file)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30592 entries, 0 to 30591
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Категория      30592 non-null  object 
 1   Название       30592 non-null  object 
 2   Страна         30483 non-null  object 
 3   Скидка         15533 non-null  object 
 4   Цена           30592 non-null  float64
 5   Цена по карте  30590 non-null  float64
dtypes: float64(2), object(4)
memory usage: 1.4+ MB


In [4]:
df['Категория'].nunique()

28